<script type="text/javascript"
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML">
</script>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>

Jakub Musiał

# **AOD - lab2**

## **Exercise 5 - Production plan**

Knowing:
* The product prices per kg
* Material costs per kg
* Production times of each product per machine
* Maximum weekly demand of each product
* Machine work time costs (per hour)

Determine the optimal weekly production plan and the sales profit

In [1]:
using JuMP
using GLPK

import JSON

#### **Data and utils**

In [2]:
# Data initialization
data_general = JSON.parse(read("data.json", String))
# Extraxt exercise 1 data from the general dictionary
data = data_general["ex5"]

Dict{String, Any} with 3 entries:
  "machine_cost_per_hour" => Dict{String, Any}("m3"=>3, "m2"=>2, "m1"=>2)
  "products"              => Dict{String, Any}("p4"=>Dict{String, Any}("kg_prod…
  "max_time"              => 3600

In [3]:
products = sort(collect(keys(data["products"])))
machines = sort(collect(keys(data["machine_cost_per_hour"])))
max_time = data["max_time"]
products, machines, max_time

(["p1", "p2", "p3", "p4"], ["m1", "m2", "m3"], 3600)

In [4]:
price(product::String) = data["products"][product]["price_per_kg"]
material_cost(product::String) = data["products"][product]["material_cost"]
demand(product::String) = data["products"][product]["max_weekly_demand"]
prod_time(product::String, machine::String) = data["products"][product]["kg_production_time"][machine]
machine_cost(machine::String) = data["machine_cost_per_hour"][machine]
profit(
    product::String
) = price(product) - material_cost(product) - sum(prod_time(product, machine) / 60 * machine_cost(machine) for machine in machines)

profit (generic function with 1 method)

#### **Build the model**

Notation:
* $P = \text{products}$
* $M = \text{machines}$

In [5]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

**Predictor variables:**

$x_{p}$ where $(p) \text{ } \epsilon \text{ } P$ - the number of produced $kg$ of product $p$

In [6]:
@variable(model, x[products] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["p1", "p2", "p3", "p4"]
And data, a 4-element Vector{VariableRef}:
 x[p1]
 x[p2]
 x[p3]
 x[p4]

**Constraints:** 

* Product ammounts must be non-negative
  
  $(\forall{(p) \text{ } \epsilon \text{ } P})(x_p \geq 0)$

* The company cannot produce more of each product than their maximum weekly demands
  
  $(\forall{p \epsilon P})( x_p \leq \text{max\_weekly\_demand(p)})$
  
* Machines cannot work for more than $60h$ per week
  
  $(\forall{m \epsilon M})(\sum_{p \epsilon P} x_p * \text{kg\_production\_time}(p, m) \leq \text{max\_machine\_work\_time} = 3600 \text{ min})$

In [7]:
@constraints(model, begin
    [p in products], sum(x[p]) <= demand(p)
    [m in machines], sum(x[p] * prod_time(p, m) for p in products) <= max_time
end)

(1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["p1", "p2", "p3", "p4"]
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[p1] <= 400.0
 x[p2] <= 100.0
 x[p3] <= 150.0
 x[p4] <= 500.0, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["m1", "m2", "m3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 5 x[p1] + 3 x[p2] + 4 x[p3] + 4 x[p4] <= 3600.0
 10 x[p1] + 6 x[p2] + 5 x[p3] +

**Objective:** maximum profit from product sales

$max(\sum_{p \text{ } \epsilon \text{ } P} x_p * profit(p))$

Where $\text{profit}(p) = \sum_{m \text{ } \epsilon \text{ } M}[\text{price}(p) - \text{material\_cost}(p) - (\frac{\text{prod\_time}(p, m)}{60} * \text{machine\_cost(m)})]$ - profit of producing a single piece of product $p$

In [8]:
@objective(model, Max, sum(x[p] * profit(p) for p in products))

4.2 x[p1] + 5.5 x[p2] + 4.55 x[p3] + 3.75 x[p4]

In [9]:
# Optimize the model
optimize!(model)
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 3.63250e+03
  Objective bound    : Inf
  Dual objective value : 3.63250e+03

* Work counters
  Solve time (sec)   : 0.00000e+00


#### **Results:**

In [15]:
println("Weekly production plan:")
total_profit = 0
machine_work_times = Dict{String, Integer}("m1"=>0, "m2"=>0, "m3"=>0)
for p in products
    println("product: ", p)
    println("\tquantity = ", value(x[p]))
    println("\tprofit = ", value(x[p]) * profit(p))
    println("\tmachine work times:")
    total_profit += value(x[p]) * profit(p)
    for m in machines
        time = value(x[p]) * prod_time(p, m)
        println("\t\t", m, time)
        machine_work_times[m] += time
    end
end

println("\nTotal profit = ", total_profit)
println("\nTotal machine work times [min]:")
for (m, t) in machine_work_times
    println("\t", m, " : ", t)
end

Weekly production plan:
product: p1
	quantity = 125.0
	profit = 525.0
	machine work times:
		m1625.0
		m21250.0
		m3750.0
product: p2
	quantity = 100.0
	profit = 550.0
	machine work times:
		m1300.0
		m2600.0
		m3400.0
product: p3
	quantity = 150.0
	profit = 682.5
	machine work times:
		m1600.0
		m2750.0
		m3450.0
product: p4
	quantity = 500.0
	profit = 1875.0
	machine work times:
		m12000.0
		m21000.0
		m3500.0

Total profit = 3632.5

Total machine work times [min]:
	m3 : 2100
	m2 : 3600
	m1 : 3525
